# A tutorial for _KNOWL_ - object-oriented knowledge ontology language

## The basics

### 1) Intro

KNOWL API allows you to work with ontology entities using the OOP paradigm (I will sometimes mistakenly refer to entities as objects, even though object in the ontology sense is different than in the OOP sense). Actual data that you work with are part of a "classical" ontology in the form of RDF triples. The RDF storage and RDF graph manipulation are done by the wonderful [RDFLib](https://rdflib.readthedocs.io/en/stable/). Most of the original API for working with RDF triples is exposed, so you can work using that paradigm as well. KNOWL only adds another layer of functionality. 

### 2) Setting up the OntologyAPI

#### Database under the hood

Under the hood, KNOWL utilizes RDFLib with RDFLib-SQLAlchemy as a triple store. Thus all the data are stored in a relational database (however, in the form of RDF triples). This means you need to first connect to one. Don't worry, however, we can use SQLite for an in-memory database or create a database file anywhere on your system without any need of setting-up a complicated RLDB. If you want to work with SQLite in Python, please make sure you have the [__sqlite3__](https://docs.python.org/3.7/library/sqlite3.html) library (most likely you do).

#### The DBConfig object

To help you work with connection configuration in various formats, there is a class called DBConfig. It shall be passed to the OntologyAPI object as a constructor argument. Creating the _default_ configuration object (i.e., without any parameters) will create some default connection to your local MySQL (or MariaDB) database. You can also create a config from a configuration file (YAML or JSON) that stores all the constructor arguments in the form of a dictionary. To do this and more you can utilize one of the config factory functions - functions of the DBConfig class itself. More on that later. Now we want to utilize the **getInMemoryConfig** function to create in-memory database.

#### First contact

In the next cell, we will create an in-memory config and instantiate an OntologyAPI object using that config.

In [1]:
from knowl import OntologyAPI, DBConfig

config = DBConfig.getInMemoryConfig()
onto = OntologyAPI(config)

The **getInMemoryConfig** method takes several arguments. You can change the name of your database (this is mostly useful when working with actual RLDB where there will be many databases). You can also change the _baseURL_ of your ontology. This serves as an identificator of your specific ontology. All entities from within this ontology will be called _baseURL#entityName_. Thus you can tell where is that entity defined just by looking at its full name. You can change any of the parameters, if you like, for now, it does not matter.

### 2) Loading and merging ontologies

If you try the above code, you will get a completely empty ontology. No classes, no other entities, nothing. Well, except for a few namespaces. This is ok if you just want to try some basics. However, if you want to try something more complex, you might want to preload some ontology into your ontology. That is, to merge some entities into your ontology.

### 3) Creating an entity

#### Entities, objects, subjects and other creatures

This mini-tutorial shall not serve as an intro to ontologies. If you would like to know more, there are many resources on the Internet (and I will create a separate tutorial for that in some distant future).
However, some basics are needed to know what is going on. Remember, any Python object you create using this API, is still an entity living in an ontology. So all things that apply to ontology entities apply to that Python object. Now, let's (attempt to) clear the terminology. I guess you know what a Python object is but I will talk about it shortly anyway. It is an instance of a Python class and they are both a part of an object-oriented programming paradigm (or OOP for short). An object has certain properties (also methods but we will treat them as actionable properties for now) imparted by its class.

This is very similar to ontology, although the terminology is a bit different. There are classes that impart certain properties upon their instances. However, in case of an ontology, the instances are usually called individuals (though class instance works just fine) and the properties are usually called predicates (though you can still use the term property where suitable; predicate is like a generalization or abstraction of a property). Classes are called classes.

Now, here comes the confusing part. There are objects in ontology as well, however, in this case, they are objects more in the sense of a language. That is, objects are entities that are, in this specific context, acted upon. Complementary to that are subjects. These are the "active" entities in that specific context - entities that act upon objects. An entity can appear in the role of a subject or an object, depending on the predicate. For example:  

> personA isParentOf personB  
> personB isParentOf personC  

Here, _personB_ is an object in case of the first clause (implying that personB is a child of personA) and a subject in case of the second clause (stating that personB is also a parent of another person - personC; logically, personA is a grandparent of personC with personB being the connection between them).
Now, for this reason, I am using the word *entity* to denote something that can be both - a subject and/or an object. An entity does not necessarily have to appear in both roles. Some entities might appear only as a subject or only as an object. This depends solely on the predicates used in the ontology. So the concept of subjects and objects is only relevant when talking about a specific predicate.

This gets us to the **triples**. A triple is a clause, axiom or an assertion (whatever terminology gets the concept through to you) consisting of a _subject_, a _predicate_, and an _object_ - in that order and no other. For simplicity and shortness, I sometimes call the _"s, p, o"_ triples. Although RDF triple is probably a more correct term, "s, p, o" triple seems like a clearer term. Therefore, I use it when it has to be absolutely clear what should happen (e.g., when describing method parameters).

#### Don't be scared but there are four elephants watching you from the corners of your room

So, there are a few peculiarities about the terminology that should be expanded a little further and that you should remember.

1) What are predicates? Weeeeeeell, think about predicates in the language sense, check out some "s, p, o" triple examples and don't ask me for a definition. Simple explanations involve using words like "properties" and "relations" and this could work most of the time. But they should be understood in a more general form. Simply put, they are the "what is going on between the subject and object" things. It is a good practice to "verbify" them (yes, that is a word; I define ontologies, what are a couple of word definitions compared to them). That is, use "hasColor", "isParent(Of)", "wasHonored", "canFly", and so on. In other words, the predicate should define an action by which the subject acts upon the object (in a more philosophical sense). This, however, is not always convenient and people often use predicates like "address", e.g.:  
> personX name Felix  
> personX address "34th Universal Street, Metropolis, Earth"  

Thus the record (set of clauses related to the person) of the personX looks like a form the person had filled-up. The correct format using the aforementioned verbification would be:  
> personX hasName Felix  
> personX hasAddress "35th Universal Street, Metropolis, Earth"  

However, I am not ontology police, so I will not say that the first case is totally wrong. If you have good reasons to use the first format (e.g, you want to automatically generate forms and "hasName" would require additional processing to desillyfy it), use it. However, if you don't, I would recommend using the second format, as it gives a slightly clearer insight of what is going on (i.e., it is more like a natural language sentence).


##### b) Objects and objects
As you may have noticed, the key problem when talking about OOP and ontology together, is the term "object". It is especially problematic since ther will be a method that is called "makeEntity" and what it does, is that it creates a Python class (called OntoEntity) instance, i.e., an object. But it also creates and entity within the ontology. Although, it could also create an instance of an ontology class (at the same time it does all the previous things). So, we will be dealing with objects that are entities and could appear as objects or subjects. Meanwhile, the objects are instances of some classes and could also be instances of classes but don't have to. Yeah, it is messy but only as long as you don't separate OOP objects and classes from ontology objects and classes. I will try to make sure it is clear enough what kind of objects and classes are we dealing with (OOP or ontology). But I hope in most cases, it will be clear from the context. Just remember, they are completely different things even though they share some similarities.

##### c) Classes, types, and labels  
Soooo, there is another issue with the terminology. But if you are more deeply familiar with Python and OOP, I guess this should be less problematic. Classes are the core concept of ontology. They create the roots, trunk, and main branches of the ontology (sort of) tree. But there is another term and that is *type*. Specifically, RDF.type (that is "type" in the RDF namespace). Since I mentioned Python, here is a neat similarity - try this code:





In [2]:
# We already imported the OnologyAPI class and made an instance of it, called "onto"
# Let's look at its "type" - Python function "type" tells us what kind of "thing" the provided parameter is
print(type(OntologyAPI))
# Now let's see what kind of thing is the instance "onto"
print(type(onto))

<class 'type'>
<class 'knowl.databaseAPI.OntologyAPI'>


Yeah, I lied, this isn't neat at all. This nomenclature isn't very helpful. But let's change it a bit.
```Python
<type 'class'>
<type 'knowl.databaseAPI.OntologyAPI'>
```
Yup, it's much better now. If you didn't notice, I "corrected" the output by swapping the terms "class" and "type". Now it reflects the ontology terminology (and frankly, this should have actually been Python's output as well but that is topic for another discussion). So, the **type** of the class _OntologyAPI_ is **class** (surprise, surprise, type of a class is "a class") and the **type** of the instance (i.e., a Python object) of the class is the class it is an instance of. Which should also make sense but to reiterate using a simpler example:

> A **chair** is a class of "things" (don't want to say the word object here but yes, in natural language we would say a class of objects). So the type of the "entity" **chair** is a **class** (of things). If you now point out a specific chair, like a chair in your office (or in your classroom, don't want to discriminate against younger audience) what would be the type for that chair? If you answered "an office chair" (or a "horrible, back-pain and buttocks numbness inducing chair" in case of the classroom chair) then you answered absolutely correctly. But, of course, these are just subclasses of the "primordial", ultra-abstract **chair**.  

So the type of a specific thing in the real world (and also in ontology, since ontology is a description of the real world) is the class of that thing. Which is congruent with the "corrected" Python example.
Here is how the chair example would look like using RDF triples (a simplified version):  

> chair type Class  
> thatChairYouAreSittingOn type chair  

I hope that this is clear now. And I also hope you forgot the was mention about labels at the beginning of this section...

##### d) The thingies in the ontology  

By this point, you have heard it all, subjects, objects, entities, things... These are all living inside your ontology and apparently, they can be many things. The question is, what should you call them? I have no idea. But, here is what I do. I call the things that could act as subjects or objects "entities". I should not say this, since entity is also an unwelcomed term, but don't use the word "concept" in relation to the things in the ontology. Well, unless you are referring to the _concept_ of something being a _class_ or having a _type_. This is mostly my opinion but these could/should be called concepts since they actually are concepts (i.e. something that "lives" inside your mind but has no physical equivalent in the real world. However, things in the database usually have physical equivalent or some other realization in the real world. For example, "a chair" is not a concept. It is a very real, physical thing (especially when someone throws it at you :O ). This is why ontologists don't like people calling things inside the ontology "concepts". As they are (mostly) not.


### Let's actually create an entity

An entity can be created in several ways. We will talk about a few of them and then maybe I will talk about more of them in future updates of this document.